## 어떠한 전처리 없이 제출
- First: 340 (1673.54867) (LinearRegression)

In [137]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error

### data load

In [138]:
df1 = pd.read_csv("../dataset/international_trade.csv")
df2 = pd.read_csv("../dataset/sample_submission.csv")
df3 = pd.read_csv("../dataset/test.csv")
df4 = pd.read_csv("../dataset/train.csv")

### 수치화

In [139]:
def categoryChange(df):
    """ 카테고리화 """
    ordinal_encoder = OrdinalEncoder()
    ordinal = ordinal_encoder.fit_transform(df)
    return ordinal

In [140]:
for name in ['item','corporation','location']:
    df4.loc[:,name] = categoryChange(df4[[name]])

In [142]:
df4

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,4.0,0.0,0.0,0.0,0.0
1,TG_A_J_20190102,2019-01-02,4.0,0.0,0.0,0.0,0.0
2,TG_A_J_20190103,2019-01-03,4.0,0.0,0.0,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,4.0,0.0,0.0,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,4.0,0.0,0.0,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,3.0,5.0,0.0,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,3.0,5.0,0.0,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,3.0,5.0,0.0,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,3.0,5.0,0.0,477220.0,523.0


### model training

In [123]:
X = df4.drop(['ID','price(원/kg)','supply(kg)','timestamp'],axis = 1)
Y = df4['price(원/kg)']

In [124]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42,stratify= df4['item'])

In [125]:
model = RandomForestRegressor()

In [126]:
model.fit(x_train,y_train)

RandomForestRegressor()

In [127]:
pred = model.predict(x_test)

In [129]:
"""
LinearRegressor -> 1904.0070927607574
RandomForestRegressor -> 1580.9442694709762
"""
forest_mse = mean_squared_error(y_test, pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

1580.9442694709762

### Submission

In [130]:
for name in ['item','corporation','location']:
    df3.loc[:,name] = categoryChange(df3[[name]])

In [131]:
X = df3.drop(['ID','timestamp'],axis = 1)

In [132]:
real = model.predict(X)

In [133]:
df2

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [134]:
df2.loc[:,'answer'] = real

In [135]:
df2['answer']

0       2664.070232
1       2664.070232
2       2664.070232
3       2664.070232
4       2664.070232
           ...     
1087     258.086039
1088     258.086039
1089     258.086039
1090     258.086039
1091     258.086039
Name: answer, Length: 1092, dtype: float64

In [136]:
df2.to_csv("answer.csv",index = False)